<a href="https://colab.research.google.com/github/Uranium1901/CS111.L21/blob/main/Sarcasm_headline_dataset/CS114_CollectHeadline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import thư viện cần thiết

In [ ]:
pip install bs4

In [ ]:
pip install requests

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

# Lấy title của 1 bài báo bất kỳ


In [ ]:
#Lấy title từ 1 bài báo
def taketitle(url):
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    for title in soup.find_all('title'):
        result=title.get_text()
    return result

# Filter lọc url cần thiết

In [ ]:
#Hàm lọc những url không cần thiết, chỉ giữ lại url bài báo của trang weeklyworldnews
#Ở website này, nhận thấy các url bài báo có chứa đúng 6 dấu '/'
def filterweeklyworldnews(url):
    count=0
    for i in range (len(url)):
        if url[i]=='/': 
            count+=1
    if count==6:
        return True
    else:
        return False

In [ ]:
#Hàm lọc những url không cần thiết, chỉ giữ lại url bài báo của trang huzlers
#Ở website này, nhận thấy các url bài báo có chứa lớn hơn hoặc bằng  3 dấu '-' bên trong url đó
def filterhuzlers(url):
    count=0
    for i in range (len(url)):
        if url[i]=='-':
            count+=1
    if count>=3:
        return True
    else:
        return False

In [ ]:
#Hàm lọc những url không cần thiết, chỉ giữ lại url bài báo của trang thehardtimes
#Ở website này, nhận thấy các url bài báo có chứa lớn hơn hoặc bằng  3 dấu '-' bên trong url đó
def filterthehardtimes(url):
    if ('thehardtimes.net' not in url) or ('the-top-5-comments' in url) : #Lọc những url không chứa thehardtimes.net, url có chứa top 5 comments
        return False
    count=0
    for i in range (len(url)):
        if url[i]=='-':
            count+=1
    if count>=3:
        return True
    else:
        return False

In [ ]:
#Hàm lọc những url không cần thiết, chỉ giữ lại url bài báo của trang weeklyworldnews
#Ở website này, nhận thấy các url bài báo có chứa đúng 7 dấu '/'
def filterthepoke(url):
  count=0
  for i in range (len(url)):
    if url[i]=='/':
      count+=1
  if count==7:
    return True
  else:
    return False

# Định dạng bộ dữ liệu

In [ ]:
# định dạng mỗi bộ dữ liệu
def format(url, title,is_sarcastic):
    data={
        "article_link": url,
        "headline": title,
        "is_sarcastic": is_sarcastic
    }
    return data

# Lấy toàn bộ các headline(title) bài báo trên từng Page của website


In [ ]:
#Lấy toàn bộ title của các bài báo trên các trang của website huzlers
URL = 'https://www.huzlers.com/page/'
data=[]
# vòng lặp các page
for page in range(1,12): 
    # Tìm kiếm toàn bộ link có trên page
    urlpage = URL + str(page) + '/'
    reqs = requests.get(urlpage)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    # loại bỏ các link giống nhau, lấy title, lưu vào mảng  
    preurl=''
    for link in soup.find_all('a'):
        if link.get('href') is not None:
            url=link.get('href')
        if filterhuzlers(url) and url!=preurl :
            title=taketitle(url)
            preurl=url
            element= format(url,title,1)
            data.append(element)

In [ ]:
URL = 'https://weeklyworldnews.com/page/'
data1=[]
for page in range(1,200):
  
    urlpage = URL + str(page) + '/'
    reqs = requests.get(urlpage)
    soup = BeautifulSoup(reqs.text, 'html.parser')
 
    preurl=''
    for link in soup.find_all('a'):
        url=link.get('href')
        if filterweeklyworldnews(url) and url!=preurl :
            temp=taketitle(url)
            # bỏ đi chuỗi - Weekly World News có trong title của bài báo
            if "- Weekly World News" in temp:
                title=temp[:temp.index('- Weekly World News')-1]
            preurl=url
            element= format(url,title,1)
            data1.append(element)

In [ ]:
URL = 'https://thehardtimes.net/page/'
data2=[]
for page in range(1,300):
  
    urlpage = URL + str(page) + '/'
    reqs = requests.get(urlpage)
    soup = BeautifulSoup(reqs.text, 'html.parser')

    preurl=''
    for link in soup.find_all('a'):
        url=link.get('href')
        if filterthehardtimes(url) and url!=preurl :
            temp=taketitle(url)
            if "- Hard Drive" in temp:
                title=temp[:temp.index('- Hard Drive')]
            else:
                title=temp
            preurl=url
            element= format(url,title,1)
            data2.append(element)

In [ ]:
URL = 'https://www.thepoke.co.uk/page/'
data3=[]
for page in range(1,600):
  
    urlpage = URL + str(page) + '/'
    reqs = requests.get(urlpage)
    soup = BeautifulSoup(reqs.text, 'html.parser')
 
    urls = []
    preurl=''
    for link in soup.find_all('a'):
        url=link.get('href')
        if filterthepoke(url) and url!=preurl :
            temp=taketitle(url)
            if "- The Poke" in temp:
              title=temp[:temp.index('- The Poke')]
            else:
              title=temp
            preurl=url
            element= format(url,title,1)
            data3.append(element)

In [ ]:
#Upload file
from google.colab import files
uploaded = files.upload()

# Ghi dữ liệu vào file

In [ ]:

with open('weeklyworldnews.json', 'w') as outfile:
    for i in data:
        json.dump(i, outfile)
        outfile.write('\n')

In [ ]:
with open('huzlers.json', 'w') as outfile:
    for i in data1:
        json.dump(i, outfile)
        outfile.write('\n')

In [ ]:
with open('thehardtimes.json', 'w') as outfile:
    for i in data2:
        json.dump(i, outfile,ensure_ascii=False)
        outfile.write('\n')

In [ ]:
with open('thepoke.json', 'w') as outfile:
    for i in data3:
        json.dump(i, outfile,ensure_ascii=False)
        outfile.write('\n')

# Download file

In [ ]:
files.download("thehardtimes.json")

In [ ]:
files.download("weeklyworldnews.json")

In [ ]:
files.download("huzlers.json")

In [ ]:
files.download("thepoke.json")